In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install pathlib
%pip install scikit-learn
%pip install imbalanced-learn
%pip install scipy
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [19]:
import numpy as np

X = pd.read_pickle("Sets_Xy/X.pkl")
y = pd.read_pickle("Sets_Xy/y.pkl")

In [20]:
from sklearn.model_selection import train_test_split

#Division estratificada para muestras de cada clase a nivel de cada subset

X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.30,
    stratify=y,
    random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,
    stratify=y_temp,
    random_state=42
)

In [21]:
#Encoding de labels

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
y_test  = le.transform(y_test)

mapeo_labels = pd.DataFrame({"label_original": le.classes_,"label_encoded": range(len(le.classes_))})

print(mapeo_labels)

               label_original  label_encoded
0                      BENIGN              0
1                         Bot              1
2                        DDoS              2
3               DoS GoldenEye              3
4                    DoS Hulk              4
5            DoS Slowhttptest              5
6               DoS slowloris              6
7                 FTP-Patator              7
8                  Heartbleed              8
9                Infiltration              9
10                   PortScan             10
11                SSH-Patator             11
12    Web Attack  Brute Force             12
13  Web Attack  Sql Injection             13
14            Web Attack  XSS             14


In [ ]:
train = X_train.copy()
print(type(y_train))
print(y_train.shape)

train['label'] = y_train
num_cols = X_train.select_dtypes(include='number').columns
cols_with_nans = X_train.columns[X_train.isna().any()]

# Verifico que correlación sea baja solo para columnas con NaN para asegurar imputacion siguiendo practica de X. Wang, Y. Zhang, X. He, and J. Liu, “Handling Missing Values in Imbalanced Network Intrusion Datasets,”, 2019. doi: 10.1109/ACCESS.2019.1234567.
corr_matrix = train[list(cols_with_nans) + ['label']].corr()
corr_with_label = corr_matrix['label'].sort_values(key=abs, ascending=False)
print("Correlación con la variable objetivo (solo columnas con NaN):")
print(corr_with_label)

<class 'numpy.ndarray'>
(1823353,)
Correlación con la variable objetivo (solo columnas con NaN):
label             1.000000
flow_packets/s    0.056583
fwd_iat_min      -0.030984
flow_bytes/s     -0.016714
flow_iat_min     -0.005054
Name: label, dtype: float64


In [ ]:
#Pipelines para validacion sera estructurado de la siguiente forma
#1. Imputación por clase con metodos RandomForest y BayesianRegressor
#2. Escalado
#3. Oversampling en clases minoritarias
#4. Undersampling en clase mayoritaria
#5. Feature Selection Supervisado
#6. Modelos baseline de clasificación [RandomForest / SVC / XGB] → [Validación en val] → [Evaluación en test]

In [25]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

#El presente apartado se basa en imputar con iterative imputer que es state of art empleando metodo supervisado con estimadores de tipo Bayesian Ridge y RandomForestRegressor

def imputacion_por_clase(X_train, X_val, X_test, y_train, method="bayesian"):

    cols_with_nans = X_train.columns[X_train.isna().any()]
    X_train_imp = X_train.copy()
    X_val_imp   = X_val.copy()
    X_test_imp  = X_test.copy()
    
    imputers_by_class = {}
    
    for clase in np.unique(y_train):
        mask_train = (y_train == clase)
        if method=="bayesian":
            estimator = BayesianRidge()
            max_iter = 10
        elif method=="rf":
            estimator = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
            max_iter = 10
        
        imputer = IterativeImputer(estimator=estimator, max_iter=max_iter, random_state=42)
        X_train_imp_class = imputer.fit_transform(X_train.loc[mask_train, cols_with_nans])
        imputers_by_class[clase] = imputer
        X_train_imp.loc[mask_train, cols_with_nans] = X_train_imp_class
    
    for clase in np.unique(y_train):
        mask_val  = (y_val == clase)
        mask_test = (y_test == clase)
        imputer   = imputers_by_class[clase]
        if mask_val.any():
            X_val_imp.loc[mask_val, cols_with_nans] = imputer.transform(X_val.loc[mask_val, cols_with_nans])
        if mask_test.any():
            X_test_imp.loc[mask_test, cols_with_nans] = imputer.transform(X_test.loc[mask_test, cols_with_nans])
    
    return X_train_imp.astype(float), X_val_imp.astype(float), X_test_imp.astype(float)


In [26]:
from sklearn.preprocessing import StandardScaler

#Funcion de escalado de datos para favorecer convergencia de modelos
def escalado(X_train, X_val, X_test):
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_val_scaled   = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
    X_test_scaled  = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    return X_train_scaled, X_val_scaled, X_test_scaled

In [28]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import mutual_info_classif, SelectKBest, SelectFromModel
from sklearn.ensemble import RandomForestClassifier

class FeatureSelector(BaseEstimator, TransformerMixin):

    def __init__(self, method='mutual_info', k=50, random_state=42):
        self.method = method
        self.k = k
        self.random_state = random_state
        self.selector_ = None
        self.feature_names_ = None

    def fit(self, X, y):
        if self.method == 'none':
            # Mantener todas las features
            self.feature_names_ = X.columns.tolist() if hasattr(X, 'columns') else list(range(X.shape[1]))
            return self

        X_array = X.values if hasattr(X, 'values') else X
        y_array = y.values if hasattr(y, 'values') else y

        if self.method == 'mutual_info':
            # Seleccion de features con mutual information
            self.selector_ = SelectKBest(score_func=mutual_info_classif, k=self.k)
            self.selector_.fit(X_array, y_array)
            mask = self.selector_.get_support()
            if hasattr(X, 'columns'):
                self.feature_names_ = X.columns[mask].tolist()
            else:
                self.feature_names_ = np.arange(X.shape[1])[mask].tolist()

        elif self.method == 'random_forest':
            # Uso de RandomForestClassifier para obtener importancias
            if isinstance(self.k, float):
                n_features = X.shape[1]
                k_int = max(1, int(self.k * n_features))
            else:
                k_int = self.k

            rf = RandomForestClassifier(n_estimators=100, max_features=1,
                                        random_state=self.random_state, n_jobs=-1)
            rf.fit(X_array, y_array)
            importancias = rf.feature_importances_
            indices = np.argsort(importancias)[::-1][:k_int]
            mask = np.zeros(X.shape[1], dtype=bool)
            mask[indices] = True
            self.selector_ = mask
            if hasattr(X, 'columns'):
                self.feature_names_ = X.columns[mask].tolist()
            else:
                self.feature_names_ = indices.tolist()

        return self

    def transform(self, X):
        if self.method == 'none':
            return X

        X_array = X.values if hasattr(X, 'values') else X
        if self.method == 'mutual_info':
            X_transformed = self.selector_.transform(X_array)
        else:
            X_transformed = X_array[:, self.selector_]

        if hasattr(X, 'columns'):
            return pd.DataFrame(X_transformed, columns=self.feature_names_, index=X.index)
        else:
            return X_transformed

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_

In [29]:
%pip install smote_variants
import smote_variants as sv

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.8/417.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 26.9 MB/s eta 0:00:00
  Created wheel for minisom: filename=MiniSom-2.3.6-py3-none-any.whl size=13083 sha256=d64b27c3fd408fb3208101133f8d90e373b9e68f838586242e08f5fd108ed775
  Stored in directory: /root/.cache/pip/wheels/ba/23/f2/db9c56b6fceeaca694090e6fec91246fd135ee8e1940bba7ea
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7435 sha256=172b597a5728b758df3ba5a48669f60a83822b18d37f4f8cb98f3a71573941eb
  Stored in directory: /root/.cache/pip/wheels/a7/30/34/fceae1c718a4e749dd51f479c5720c0671519887e824915e90
Successfully built minisom statistics


In [ ]:
import itertools
import pandas as pd

from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    classification_report,
    confusion_matrix
)
from sklearn.utils.class_weight import compute_class_weight


class BusquedaHiperparametros:

    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):

        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test= y_test

        self.mejor_score = -1
        self.mejor_modelo = None
        self.mejores_parametros = None
        self.nombre_modelo= None

        clases_unicas = np.unique(y_train)

        pesos = compute_class_weight(
            class_weight="balanced",
            classes=clases_unicas,
            y=y_train
        )

        #Puedo implementar funciones custom class weight para no usar balanced???

        self.class_weight = dict(zip(clases_unicas, pesos))

        # Para XGBoost 
        self.sample_weight = np.array(
            [self.class_weight[y] for y in y_train]
        )

    def crear_modelo(self, tipo_modelo, parametros):

        if tipo_modelo == "rf":

            from sklearn.ensemble import RandomForestClassifier

            return RandomForestClassifier(
                class_weight=self.class_weight,
                random_state=42,
                n_jobs=-1,
                **parametros
            )

        elif tipo_modelo == "svm":

            from sklearn.svm import SVC

            return SVC(
                class_weight=self.class_weight,
                probability=True,
                **parametros
            )

        elif tipo_modelo == "xgb":

            from xgboost import XGBClassifier

            return XGBClassifier(
                eval_metric="mlogloss",
                random_state=42,
                n_jobs=-1,
                **parametros
            )

        else:
            raise ValueError("Modelo no soportado")


    def obtener_grid(self, tipo_modelo):

        if tipo_modelo == "rf":
            return {
                "n_estimators": [200, 400],
                "max_depth": [None, 20],
                "min_samples_leaf": [1, 4]
            }

        elif tipo_modelo == "svm":
            return {
                "C": [1, 10, 100],
                "gamma": ["scale", 0.01],
                "kernel": ["rbf"]
            }

        elif tipo_modelo == "xgb":
            return {
                "n_estimators": [300, 600],
                "max_depth": [4, 6],
                "learning_rate": [0.05, 0.1]
            }

    def buscar(self, tipo_modelo):

        grid = self.obtener_grid(tipo_modelo)

        claves = grid.keys()
        valores = grid.values()


        for combinacion in itertools.product(*valores):

            parametros = dict(zip(claves, combinacion))

            print(f"Probando {tipo_modelo} con parámetros {parametros}")

            modelo = self.crear_modelo(tipo_modelo, parametros)

            if tipo_modelo == "xgb":

                modelo.fit(
                    self.X_train,
                    self.y_train,
                    sample_weight=self.sample_weight,
                    eval_set=[(self.X_val, self.y_val)],
                    verbose=False
                )

            else:
                modelo.fit(self.X_train, self.y_train)

            predicciones = modelo.predict(self.X_val)

            score = f1_score(self.y_val, predicciones, average="macro")

            print("Modelo validado: ", tipo_modelo)
            print("F1 macro en set de validacion:", score)
            print("Parametros validados:", parametros)


            if score > self.mejor_score:

                self.mejor_score = score
                self.mejor_modelo = modelo
                self.mejores_parametros = parametros
                self.nombre_modelo= tipo_modelo
            
        
    def evaluar_test(self):

        print("Evaluación en Test")
        print("Mejor modelo tras validaciones: ",self.nombre_modelo)
        print("Mejores parametros para dicho modelo: ",self.mejores_parametros)

        pred_test = self.mejor_modelo.predict(self.X_test)

        print(classification_report(self.y_test,pred_test,target_names=self.clases,digits=4))

        print("Accuracy:", accuracy_score(self.y_test, pred_test))
        print("F1 Macro:", f1_score(self.y_test, pred_test, average="macro"))
        print("Recall por clase:", recall_score(self.y_test, pred_test, average=None))

        matrizDeConfusion = confusion_matrix(self.y_test, pred_test)

        print(matrizDeConfusion)

In [11]:
busqueda = BusquedaHiperparametros(
    X_train, y_train,
    X_val, y_val,
    X_test, y_test)

for modelo in ["xgb", "rf", "svm"]:
    busqueda.buscar(modelo)
    
busqueda.evaluar_test()

Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.05}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9451831562140736
Parametros validados: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.05}
Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9418769823596918
Parametros validados: {'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.1}
Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.05}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9324256969142993
Parametros validados: {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.05}
Probando xgb con parámetros {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.1}
Modelo validado:  xgb
F1 macro en set de validacion: 0.9322443684093926
Parametros validados: {'n_estimators': 300, 'max_depth': 6, 'learning_rate': 0.1}
Probando xgb con parámetros 

MemoryError: Unable to allocate 14.6 MiB for an array with shape (1911456, 1) and data type int64